In [7]:
from load_jsons import load_jsons
import pandas as pd
import matplotlib.pyplot as plt
from random import randint
from json import load

In [3]:
def rebuild(data:list[dict]) -> list[dict]:
    out = []
    for datapoint in data:
        solver = datapoint["solver"]
        if solver == "or-tools":
            if "--threads=8" in datapoint["solverOptions"][0]:
                continue
            else:
                solver = "or-tools-1"
        out.append({
            "instance": datapoint["essenceParams"][0],
            "model": datapoint["useExistingModels"][0].split("/")[-1],
            "solver": solver,
            "total_time": datapoint["totalTime"] if datapoint["status"] == "OK" else 36000,
            "status": datapoint["status"],
            "solver_options": datapoint["solverOptions"][0]
        })
    return out

In [15]:
data = load_jsons("../../EssenceCatalog-runs/problems/csplib-prob001-CarSequencing/", verbose=False)
data = rebuild(data)

In [16]:
instances_data = {}
for datapoint in data:
    inst = datapoint["instance"]
    model = datapoint["model"]
    solver = datapoint["solver"]
    comb = f"{solver}_{model}"
    if not inst in instances_data:
        instances_data[inst] = {}
    instances_data[inst][comb] = datapoint["total_time"]

In [18]:
def load_file(file_name):
    with open(file_name) as f:
        return f.read()

def get_dataset(data):
    dataset = []
    for key in data.keys():
        datapoint = data[key]
        combinations = list(datapoint.keys())
        best = {'combination': combinations[0], 'time': datapoint[combinations[0]]}
        times = []
        for comb in combinations:
            if "bin" in comb:
                continue
            times.append({"time": datapoint[comb], "combination": comb})
            if best['time'] > datapoint[comb]:
                best = {'combination': comb, 'time': datapoint[comb]}

        instance = key.replace("/podmandir/EssenceCatalog/problems/csplib-prob001-CarSequencing/", "")
        dataset.append({
            "combination": best["combination"],
            "time": best["time"],
            "instance_value":load_file(f"../{instance}"),
            "instance_name": instance,
            "all_times": times
        })
    return dataset

dataset = get_dataset(instances_data)

In [20]:
from json import dump
f = open("../data/datasets/dataset_CarSequencing-2024-03-19.json","w")
dump(dataset, f)
f.close()